# Table of contents


> 0. [Minimum required packages and settings](#0)
>>
>> 0.1: [import required packages](#0.1)
>
> 1. [Extract region sequences](#1)

<a id='0.1'></a>
## 0.1 load required packages

In [2]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

2022-01-16 16:18:16,783 [INFO] WRITING LOG OUTPUT TO C:\Users\shiwei\.cellpose\run.log
34428


<a id='1'></a>
# 1 Extract region sequences

In [3]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers'

In [4]:
resolution = 0
flanking = 10000
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mop_SE_1000')
if not os.path.exists(library_folder):
    print(f"create library folder: {library_folder}")
    os.makedirs(library_folder)
# folder for fasta sequences
sequence_folder = os.path.join(library_folder, 'sequences')
if not os.path.exists(sequence_folder):
    print(f"create sequence folder: {sequence_folder}")
    os.makedirs(sequence_folder)
# folder to save result probes
report_folder = os.path.join(library_folder, 'reports')
if not os.path.exists(report_folder):
    print(f"create report folder: {report_folder}")
    os.makedirs(report_folder)
    
print(f"-- library_folder: {library_folder}")
print(f"-- sequence_folder: {sequence_folder}")
print(f"-- report_folder: {report_folder}")

-- library_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000
-- sequence_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences
-- report_folder: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports


## 1.1 select regions

In [5]:
from tqdm.notebook import tqdm

In [13]:
import pandas as pd

sel_SE_df = pd.read_excel(library_folder+os.sep+'merged_selected_SE_all_cell_types.xlsx', index_col=None)

In [59]:
sel_SE_df.head()

sel_SE_df_chrX = sel_SE_df[984:]
sel_SE_df_chrN = sel_SE_df[:984]

In [71]:
sel_SE_df_chrN['sort_CHROM'] = [int(i.split('chr')[-1]) for i in sel_SE_df_chrN['CHROM']]
sel_SE_df_chrX['sort_CHROM'] = 20

sel_SE_df_sort_chrN = sel_SE_df_chrN.sort_values(by =['sort_CHROM', 'merged_START'])
sel_SE_df_sort_chrX = sel_SE_df_chrX.sort_values(by =['sort_CHROM', 'merged_START'])
#sel_SE_df_sort[100:200]

sel_SE_df_sort_all = pd.concat([sel_SE_df_sort_chrN, sel_SE_df_sort_chrX])
sel_SE_df_new = sel_SE_df_sort_all.reset_index()

sel_SE_df_new['region_name'] = sel_SE_df_new.index +1
#sel_SE_df_new[:100]



<ipython-input-71-438bec6918df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_SE_df_chrN['sort_CHROM'] = [int(i.split('chr')[-1]) for i in sel_SE_df_chrN['CHROM']]
<ipython-input-71-438bec6918df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_SE_df_chrX['sort_CHROM'] = 20


In [67]:
sel_SE_df_new.to_excel(library_folder+os.sep+'selected_merged_SE_list.xlsx', index=False)

In [78]:
sel_SE_df_new.head()

sel_SE_df_new[70:80]

,index,CHROM,merged_START,merged_STOP,merged_CELL_TYPE,merged_initial_index,score,ave_score,merged_SIZE,sort_CHROM,region_name
70,70,chr1,187645279,187694850,['Pvalb'],[17244],[1.],1.000000,49571,1,71
71,71,chr1,189017529,189066052,"['L4_5_IT', 'L6_CT', 'L5_6_NP']","[4357, 11167, 5708]",[0.7846 0.98129 0.90823],0.891373,48523,1,72
72,72,chr1,189185060,189229230,['Astro'],[601],[1.],1.000000,44170,1,73
73,73,chr1,189490922,189533419,['L5_IT'],[7163],[1.],1.000000,42497,1,74
74,74,chr1,189876129,189928237,['Pvalb'],[17149],[1.],1.000000,52108,1,75
75,75,chr1,193085619,193123645,"['L4_5_IT', 'L5_IT']","[3426, 6692]",[1. 0.97005],0.985025,38026,1,76
76,515,chr2,3139373,3173216,['L5_PT'],[8452],[1.],1.000000,33843,2,77
77,516,chr2,4036085,4073325,['Astro'],[282],[1.],1.000000,37240,2,78
78,517,chr2,5161601,5200127,"['L6_IT', 'L4_5_IT']","[12594, 4307]",[0.93911 0.99842],0.968765,38526,2,79
79,518,chr2,11319886,11375319,['Oligo'],[15952],[1.],1.000000,55433,2,80


In [113]:
sel_SE_df_for_bed =sel_SE_df_new[['CHROM','merged_START','merged_STOP','region_name']]

#sel_SE_df_for_bed['merged_START_str']=[str(i) for i in sel_SE_df_for_bed['merged_START']]
#sel_SE_df_for_bed['merged_STOP_str']=[str(i) for i in sel_SE_df_for_bed['merged_STOP']]
#sel_SE_df_for_bed['region_name_str']=[str(i) for i in sel_SE_df_for_bed['region_name']]

#sel_SE_df_for_bed = sel_SE_df_for_bed[['CHROM','merged_START_str','merged_STOP_str','region_name_str']]
sel_SE_df_for_bed

,CHROM,merged_START,merged_STOP,region_name
0,chr1,9610471,9646458,1
1,chr1,9759317,9835402,2
2,chr1,20898605,20943251,3
3,chr1,21485446,21554433,4
4,chr1,21684557,21783163,5
...,...,...,...,...
986,chrX,101920335,102005609,987
987,chrX,136782307,136841006,988
988,chrX,140491839,140537985,989
989,chrX,167132110,167181320,990


In [124]:
# generate a 'bed' file from the processed dataframe

region_file = os.path.join(library_folder, 'Seleted_merged_super_enhancer_list.bed')
import csv
with open(region_file, 'w') as _reg_file:
    
    #tsv_writer = csv.writer(_reg_file, delimiter='\t')
    #new_line_writer = csv.writer(_reg_file, delimiter='\n')
    df_columns = list(sel_SE_df_for_bed.columns)

    for _peak in sel_SE_df_for_bed.iloc():
        for  _col in df_columns:
            _reg_file.write(str(_peak[_col]))
            _reg_file.write('\t')
        _reg_file.write('\n')
         

In [125]:
reload(library_tools.sequences)
region_file = os.path.join(library_folder, 'Seleted_merged_super_enhancer_list.bed')
region_dict = library_tools.sequences.read_region_file(region_file)
# add strand information
for _i in range(len(region_dict)):
    region_dict[_i]['Strand'] = '+'
records = library_tools.sequences.extract_sequence(region_dict, genome_folder, 
                                                   resolution=resolution,
                                                   flanking=flanking,
                                                   save_folder=sequence_folder)

Input region file is: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\Seleted_merged_super_enhancer_list.bed
- 991 regions loaded from file: Seleted_merged_super_enhancer_list.bed
-- searching among 23 references
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musc

-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.1.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Li

-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.2.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Li

-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.3.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Li

-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Genome\Mus_musculus.GRCm38.dna.chromosome.4.fa.
-- a match found in file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Li

...

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\681_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\682_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\683_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\684_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\685_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\686_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\687_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\749_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\750_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\751_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\752_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\753_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\754_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\755_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\818_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\819_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\820_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\821_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\822_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\823_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\824_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\888_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\889_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\890_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\891_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\892_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\893_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\894_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\954_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\955_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\956_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\957_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\958_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\959_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\960_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2

In [126]:
# calculate num of regions in each cluster
chr_2_region_num = {}
for _reg_info in region_dict:
    _chr_name = _reg_info['Chr']
    if 'chr' in _chr_name:
        _chr = _chr_name.split('chr')[1]
    else:
        _chr = _chr_name
    if _chr not in chr_2_region_num:
        chr_2_region_num[_chr] = 0
    chr_2_region_num[_chr] += 1
# save
pickle.dump(chr_2_region_num, open(os.path.join(library_folder,'chr_2_region_num.pkl'), 'wb'))

In [127]:
chr_2_region_num

{'1': 76,
 '2': 75,
 '3': 60,
 '4': 63,
 '5': 60,
 '6': 57,
 '7': 45,
 '8': 53,
 '9': 56,
 '10': 65,
 '11': 57,
 '12': 36,
 '13': 49,
 '14': 57,
 '15': 35,
 '16': 38,
 '17': 34,
 '18': 33,
 '19': 35,
 'X': 7}

In [128]:
region_dict

[{'Chr': 'chr1',
  'Start': '9610471',
  'End': '9646458',
  'Name': '1',
  'Score': '',
  'Gene': '1',
  'Region': '1:9610471-9646458',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '9759317',
  'End': '9835402',
  'Name': '2',
  'Score': '',
  'Gene': '2',
  'Region': '1:9759317-9835402',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '20898605',
  'End': '20943251',
  'Name': '3',
  'Score': '',
  'Gene': '3',
  'Region': '1:20898605-20943251',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '21485446',
  'End': '21554433',
  'Name': '4',
  'Score': '',
  'Gene': '4',
  'Region': '1:21485446-21554433',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '21684557',
  'End': '21783163',
  'Name': '5',
  'Score': '',
  'Gene': '5',
  'Region': '1:21684557-21783163',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '23255924',
  'End': '23287215',
  'Name': '6',
  'Score': '',
  'Gene': '6',
  'Region': '1:23255924-23287215',
  'Strand': '+'},
 {'Chr': 'chr1',
  'Start': '23928055',
  'End': '2398

<a id='2'></a>
# 2. Design probe targeting sequences by probe_designer

run probe_designer, remeber to clear the memory usage because each table should take ~32GB

<a id='2.1'></a>
## 2.1 Construct count table with all the 17-mers in the genome

Only do this if you don't have pre-built 17-mer

However you can do almost the same thing for your own library during quality check

This library requires mm10 genome

In [130]:
overwrite_table = False

### construct map for whole genome

In [131]:
from ImageAnalysis3 import library_tools

In [132]:
reload(library_tools.design)
 
genome_table_file = os.path.join(reference_folder, 'GRCm38_genome_17w.npy')

if not os.path.exists(genome_table_file) or overwrite_table:
    # genome
    _genome_filenames = [os.path.join(genome_folder, _fl) 
         for _fl in os.listdir(genome_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_genome_filenames))

    ct = library_tools.design.countTable(word=17,save_file=genome_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_genome_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()

    # clear RAM if contructed countable 
    del(ct)
else:
    print(f"reference table: {genome_table_file} already exist, skip.")

reference table: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\GRCm38_genome_17w.npy already exist, skip.


### construct map for transcriptome

In [134]:
from tqdm import tqdm
# transcriptome
transcriptome_folder = os.path.join(reference_folder, 'Transcriptome')

transcriptome_table_file = os.path.join(reference_folder, 'GRCm38_transcriptome_17w.npy')

if not os.path.exists(transcriptome_table_file) or overwrite_table:
    # transcriptome
    _transcriptome_filenames = [os.path.join(transcriptome_folder, _fl) 
         for _fl in os.listdir(transcriptome_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_transcriptome_filenames))
    
    ct = library_tools.design.countTable(word=17,save_file=transcriptome_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_transcriptome_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()
    
    # clear RAM if contructed countable 
    del(ct)
else:
    print(f"reference table: {transcriptome_table_file} already exist, skip.")

reference table: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\GRCm38_transcriptome_17w.npy already exist, skip.


### construct map for repeats from RepBase

In [135]:
from tqdm import tqdm
# repeat
repeat_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\Repbase'

repeat_table_file = os.path.join(reference_folder, 'Repbase_v2603_repeat_17w.npy')

if not os.path.exists(repeat_table_file) or overwrite_table:
    # repeat
    _repeat_filenames = [os.path.join(repeat_folder, _fl) 
         for _fl in os.listdir(repeat_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_repeat_filenames))
    
    ct = library_tools.design.countTable(word=17,save_file=repeat_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_repeat_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()
    
    # clear RAM if contructed countable 
    del(ct)
else:
    print(f"reference table: {repeat_table_file} already exist, skip.")

reference table: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl\Repbase_v2603_repeat_17w.npy already exist, skip.


<a id='2.2'></a>
## 2.2 Design probes

In [137]:
# requires pre_defined genome_folder and library_folder
# Indices
genome_index = os.path.join(reference_folder, 'GRCm38_genome_17w.npy')
transcriptome_index = os.path.join(reference_folder, 'GRCm38_transcriptome_17w.npy') 
repeat_index = os.path.join(reference_folder, 'Repbase_v2603_repeat_17w.npy')
#ref_merfish_index = os.path.join(reference_folder, 'M1_meng_MERFISH_17w.npy') # merfish designed by Meng
# get input files 
input_files = glob.glob(os.path.join(sequence_folder, '*.fasta'))

print(f"{len(input_files)} regions loaded to design probes.")

if not os.path.exists(report_folder):
    os.makedirs(report_folder)
    
# filename to save probe reports
save_file = os.path.join(report_folder, f'merged_probes.pbr')
print(save_file)

991 regions loaded to design probes.
\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr


### create pb_designer class

In [5]:
save_file

NameError: name 'save_file' is not defined

In [144]:
reload(library_tools)
reload(library_tools.design)

pb_designer = library_tools.design.pb_reports_class(
    sequence_dic={'file':input_files,
                  'rev_com':True, # design two strands
                  'two_stranded':True},
    map_dic={'genome':{'file':genome_index,'rev_com':False,'two_stranded':True},
             'transcriptome':{'file':transcriptome_index,'rev_com':True,'two_stranded':False},
             'rep_genome':{'file':repeat_index,'rev_com':False,'two_stranded':True},
             'self_sequences':{'file':input_files,'force_list':True,'rev_com':False,'two_stranded':True},
             #'ref_merfish':{'file':ref_merfish_index,'rev_com':False,'two_stranded':True},
             },
        save_file=save_file,
    params_dic={'word_size':17,'pb_len':42,'buffer_len':2,'max_count':2**16-1,
                'check_on_go': False, # whether automatically check probes
                'auto': False, # whether automatically convert reference maps
               },
    check_dic={('genome','self_sequences'): 25,
               'rep_genome': 0,
               'transcriptome': 14,
               #'ref_merfish': 14,
               'gc':[0.25,0.85],'tm': 37+0.62*50+5,
               }
    )
print(pb_designer)
#pb_designer.load_from_file(load_probes_only=True)


Probe designer derived from Bogdan Bintu:
https://github.com/BogdanBintu/ChromatinImaging/blob/master/LibraryDesign/LibraryDesigner.py
by Pu Zheng, 2020.11

Major changes:
    1. allow design of two strands
    2. separate reverse_complement (rev_com) and from two strands (two_stranded) as 
    two different inputs for map_dic and sequence_dic
    3. replace 'local_genome' with 'self_sequences' to be more explicit, and only 
    exclude the counts for the corresponding self_sequence within each input. 

Key information:
    - number of input_sequence(s): 991
    - save_file location: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr



In [140]:
pb_designer.load_from_file(load_probes_only=True)

- Fail to load from savefile: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr, file doesn't exist.


False

### calculate probe reports

In [145]:
reload(library_tools)
reload(library_tools.design)
pb_designer.computeOTmaps() # load the tables 
pb_designer.compute_pb_report() # design candidate probes

-- setting attribute: map_genome
--- finish map_genome in 97.122s.
-- setting attribute: map_transcriptome
--- finish map_transcriptome in 128.480s.
-- setting attribute: map_rep_genome
--- finish map_rep_genome in 131.638s.
Time(s): 357.40871381759644
- Designing targeting sequence for 991 regions
-- designing region: 1:9600471-9656458_:+_gene_1 region: 0, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\1_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.566s.
in 33.926s.
-- designing region: 1:9749317-9845402_:+_gene_2 region: 1, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\2_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.364s.
in 57.170s.
-- designing region: 1:20888605-20953251_:+_gene_3 region: 2, input file: \\10.245.7

--- finish map_self_sequences in 1.155s.
in 70.442s.
-- designing region: 1:60828959-60878677_:+_gene_27 region: 26, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\27_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.278s.
in 34.064s.
-- designing region: 1:62890963-62958089_:+_gene_28 region: 27, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\28_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.615s.
in 47.481s.
-- designing region: 1:68028967-68171164_:+_gene_29 region: 28, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\29_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 1.662s.
in 95.093s.
-- desig

in 46.661s.
-- designing region: 1:140354959-140541768_:+_gene_53 region: 52, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\53_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.465s.
in 110.989s.
-- designing region: 1:143861617-143934455_:+_gene_54 region: 53, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\54_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.247s.
in 43.824s.
-- designing region: 1:149846612-149908891_:+_gene_55 region: 54, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\55_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.271s.
in 37.511s.
-- designing region: 1:152397309-152486632

--- finish map_self_sequences in 0.269s.
in 35.401s.
-- designing region: 2:11309886-11385319_:+_gene_80 region: 79, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\80_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.258s.
in 45.421s.
-- designing region: 2:14897328-14968889_:+_gene_81 region: 80, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\81_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.298s.
in 43.399s.
-- designing region: 2:16513406-16660259_:+_gene_82 region: 81, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\82_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.415s.
in 87.189s.
-- desig

in 49.118s.
-- designing region: 2:77277948-77367263_:+_gene_106 region: 105, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\106_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.307s.
in 53.479s.
-- designing region: 2:83704396-83781664_:+_gene_107 region: 106, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\107_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.284s.
in 50.454s.
-- designing region: 2:84515475-84567580_:+_gene_108 region: 107, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\108_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.216s.
in 31.584s.
-- designing region: 2:84845394-84900174

...

--- finish map_self_sequences in 126.568s.
in 610.312s.
-- designing region: 17:15361839-15397147_:+_gene_886 region: 885, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\886_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.426s.
in 25.305s.
-- designing region: 17:23516967-23561137_:+_gene_887 region: 886, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\887_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.522s.
in 43.893s.
-- designing region: 17:24038288-24101203_:+_gene_888 region: 887, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\888_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.925s.
in 45

--- finish map_self_sequences in 0.320s.
in 50.556s.
-- designing region: 17:84464454-84530370_:+_gene_912 region: 911, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\912_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.288s.
in 43.881s.
-- designing region: 17:84850274-84953300_:+_gene_913 region: 912, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\913_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.861s.
in 68.187s.
-- designing region: 17:89322309-89377903_:+_gene_914 region: 913, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\914_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.275s.
in 35.44

--- finish map_self_sequences in 0.422s.
in 75.989s.
-- designing region: 18:55993891-56082519_:+_gene_938 region: 937, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\938_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.351s.
in 57.205s.
-- designing region: 18:57239664-57307027_:+_gene_939 region: 938, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\939_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.337s.
in 44.406s.
-- designing region: 18:59180021-59350310_:+_gene_940 region: 939, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\940_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 1.237s.
in 109.5

--- finish map_self_sequences in 0.315s.
in 42.700s.
-- designing region: 19:24600840-24687705_:+_gene_964 region: 963, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\964_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.329s.
in 56.232s.
-- designing region: 19:24944141-25013094_:+_gene_965 region: 964, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\965_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.297s.
in 44.570s.
-- designing region: 19:27524144-27591990_:+_gene_966 region: 965, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\966_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.299s.
in 43.43

--- finish map_self_sequences in 0.267s.
in 43.546s.
-- designing region: X:167122110-167191320_:+_gene_990 region: 989, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\990_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.798s.
in 44.749s.
-- designing region: X:169962881-170032477_:+_gene_991 region: 990, input file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\sequences\991_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.263s.
in 40.320s.
- Save reports into file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr


### check probes

In [146]:
pbs, pb_scores = pb_designer.check_probes()

-- check region:0 1:9600471-9656458_:+_gene_1, 109986 candidate probes
--- 41731 probes passed check_dic selection.
finish in 4.106s, 660 probes kept.
-- check region:1 1:9749317-9845402_:+_gene_2, 181558 candidate probes
--- 48238 probes passed check_dic selection.
finish in 5.606s, 865 probes kept.
-- check region:2 1:20888605-20953251_:+_gene_3, 126162 candidate probes
--- 50944 probes passed check_dic selection.
finish in 4.881s, 814 probes kept.
-- check region:3 1:21475446-21564433_:+_gene_4, 171916 candidate probes
--- 46142 probes passed check_dic selection.
finish in 5.386s, 809 probes kept.
-- check region:4 1:21674557-21793163_:+_gene_5, 226476 candidate probes
--- 59478 probes passed check_dic selection.
finish in 6.990s, 1057 probes kept.
-- check region:5 1:23245924-23297215_:+_gene_6, 98478 candidate probes
--- 30178 probes passed check_dic selection.
finish in 3.264s, 525 probes kept.
-- check region:6 1:23918055-23999346_:+_gene_7, 155324 candidate probes
--- 47834 pro

-- check region:53 1:143861617-143934455_:+_gene_54, 118086 candidate probes
--- 39151 probes passed check_dic selection.
finish in 3.989s, 638 probes kept.
-- check region:54 1:149846612-149908891_:+_gene_55, 123292 candidate probes
--- 43249 probes passed check_dic selection.
finish in 4.288s, 704 probes kept.
-- check region:55 1:152397309-152486632_:+_gene_56, 161488 candidate probes
--- 63119 probes passed check_dic selection.
finish in 5.974s, 933 probes kept.
-- check region:56 1:155665230-155748254_:+_gene_57, 163478 candidate probes
--- 74249 probes passed check_dic selection.
finish in 6.444s, 1060 probes kept.
-- check region:57 1:161214625-161297711_:+_gene_58, 162006 candidate probes
--- 61242 probes passed check_dic selection.
finish in 5.789s, 960 probes kept.
-- check region:58 1:162012959-162104685_:+_gene_59, 165712 candidate probes
--- 63630 probes passed check_dic selection.
finish in 6.001s, 993 probes kept.
-- check region:59 1:162655058-162776885_:+_gene_60, 2321

--- 63588 probes passed check_dic selection.
finish in 7.443s, 988 probes kept.
-- check region:106 2:83704396-83781664_:+_gene_107, 148424 candidate probes
--- 42261 probes passed check_dic selection.
finish in 5.998s, 747 probes kept.
-- check region:107 2:84515475-84567580_:+_gene_108, 94862 candidate probes
--- 33504 probes passed check_dic selection.
finish in 4.010s, 526 probes kept.
-- check region:108 2:84845394-84900174_:+_gene_109, 108422 candidate probes
--- 52903 probes passed check_dic selection.
finish in 5.292s, 736 probes kept.
-- check region:109 2:85037645-85079130_:+_gene_110, 0 candidate probes
--- 0 probes passed check_dic selection.
finish in 0.016s, 0 probes kept.
-- check region:110 2:85037645-85079130_:+_gene_111, 81532 candidate probes
--- 38862 probes passed check_dic selection.
finish in 3.977s, 586 probes kept.
-- check region:111 2:85198480-85295653_:+_gene_112, 187440 candidate probes
--- 62384 probes passed check_dic selection.
finish in 7.673s, 1002 pro

--- 48403 probes passed check_dic selection.
finish in 4.949s, 775 probes kept.
-- check region:158 3:27430699-27527522_:+_gene_159, 190666 candidate probes
--- 92764 probes passed check_dic selection.
finish in 8.111s, 1299 probes kept.
-- check region:159 3:27639981-27714878_:+_gene_160, 148402 candidate probes
--- 70414 probes passed check_dic selection.
finish in 6.265s, 977 probes kept.
-- check region:160 3:27985084-28070214_:+_gene_161, 152970 candidate probes
--- 51581 probes passed check_dic selection.
finish in 5.554s, 832 probes kept.
-- check region:161 3:33340421-33427551_:+_gene_162, 168186 candidate probes
--- 54703 probes passed check_dic selection.
finish in 6.024s, 875 probes kept.
-- check region:162 3:35074184-35113818_:+_gene_163, 78584 candidate probes
--- 38901 probes passed check_dic selection.
finish in 3.388s, 532 probes kept.
-- check region:163 3:35313296-35364803_:+_gene_164, 101910 candidate probes
--- 49457 probes passed check_dic selection.
finish in 4.5

--- 38512 probes passed check_dic selection.
finish in 4.621s, 656 probes kept.
-- check region:210 3:157139588-157226143_:+_gene_211, 171974 candidate probes
--- 66129 probes passed check_dic selection.
finish in 6.789s, 1025 probes kept.
-- check region:211 4:3040005-3100987_:+_gene_212, 87872 candidate probes
--- 7145 probes passed check_dic selection.
finish in 2.501s, 209 probes kept.
-- check region:212 4:3471494-3540953_:+_gene_213, 131520 candidate probes
--- 10270 probes passed check_dic selection.
finish in 3.508s, 283 probes kept.
-- check region:213 4:6346853-6400453_:+_gene_214, 104806 candidate probes
--- 35117 probes passed check_dic selection.
finish in 3.840s, 593 probes kept.
-- check region:214 4:8546598-8621298_:+_gene_215, 146210 candidate probes
--- 42983 probes passed check_dic selection.
finish in 5.022s, 717 probes kept.
-- check region:215 4:9621515-9668472_:+_gene_216, 93616 candidate probes
--- 35426 probes passed check_dic selection.
finish in 3.572s, 560 p

...

finish in 3.355s, 527 probes kept.
-- check region:783 14:61793570-61846789_:+_gene_784, 103368 candidate probes
--- 38091 probes passed check_dic selection.
finish in 3.763s, 581 probes kept.
-- check region:784 14:62163770-62246386_:+_gene_785, 161832 candidate probes
--- 66173 probes passed check_dic selection.
finish in 6.020s, 970 probes kept.
-- check region:785 14:62834246-62950869_:+_gene_786, 230488 candidate probes
--- 86381 probes passed check_dic selection.
finish in 8.192s, 1369 probes kept.
-- check region:786 14:63118357-63175661_:+_gene_787, 113290 candidate probes
--- 47036 probes passed check_dic selection.
finish in 4.420s, 712 probes kept.
-- check region:787 14:65504499-65594282_:+_gene_788, 177746 candidate probes
--- 81892 probes passed check_dic selection.
finish in 7.028s, 1117 probes kept.
-- check region:788 14:65960189-65998537_:+_gene_789, 75410 candidate probes
--- 34060 probes passed check_dic selection.
finish in 3.041s, 503 probes kept.
-- check region:

--- 42538 probes passed check_dic selection.
finish in 4.053s, 673 probes kept.
-- check region:835 15:83364966-83464933_:+_gene_836, 196066 candidate probes
--- 86624 probes passed check_dic selection.
finish in 7.584s, 1230 probes kept.
-- check region:836 15:88479865-88547648_:+_gene_837, 133094 candidate probes
--- 57831 probes passed check_dic selection.
finish in 5.179s, 814 probes kept.
-- check region:837 15:88818050-88864719_:+_gene_838, 91738 candidate probes
--- 51225 probes passed check_dic selection.
finish in 4.095s, 648 probes kept.
-- check region:838 15:92180253-92287287_:+_gene_839, 209906 candidate probes
--- 70945 probes passed check_dic selection.
finish in 7.040s, 1154 probes kept.
-- check region:839 15:93482810-93529982_:+_gene_840, 91854 candidate probes
--- 39949 probes passed check_dic selection.
finish in 3.624s, 572 probes kept.
-- check region:840 15:95034078-95096462_:+_gene_841, 118548 candidate probes
--- 46727 probes passed check_dic selection.
finish 

--- 25787 probes passed check_dic selection.
finish in 2.904s, 400 probes kept.
-- check region:887 17:24038288-24101203_:+_gene_888, 122354 candidate probes
--- 53165 probes passed check_dic selection.
finish in 4.796s, 786 probes kept.
-- check region:888 17:31799132-31853772_:+_gene_889, 108230 candidate probes
--- 57354 probes passed check_dic selection.
finish in 4.680s, 784 probes kept.
-- check region:889 17:34090951-34145951_:+_gene_890, 108042 candidate probes
--- 45909 probes passed check_dic selection.
finish in 4.286s, 729 probes kept.
-- check region:890 17:35516761-35560061_:+_gene_891, 84700 candidate probes
--- 33055 probes passed check_dic selection.
finish in 3.153s, 502 probes kept.
-- check region:891 17:43011066-43103508_:+_gene_892, 181472 candidate probes
--- 75593 probes passed check_dic selection.
finish in 6.782s, 1126 probes kept.
-- check region:892 17:44188990-44272216_:+_gene_893, 164168 candidate probes
--- 75739 probes passed check_dic selection.
finish 

--- 52697 probes passed check_dic selection.
finish in 4.850s, 796 probes kept.
-- check region:939 18:59180021-59350310_:+_gene_940, 331026 candidate probes
--- 97531 probes passed check_dic selection.
finish in 10.439s, 1657 probes kept.
-- check region:940 18:63719174-63787284_:+_gene_941, 135330 candidate probes
--- 39824 probes passed check_dic selection.
finish in 4.365s, 688 probes kept.
-- check region:941 18:64575603-64648451_:+_gene_942, 143296 candidate probes
--- 56496 probes passed check_dic selection.
finish in 5.337s, 810 probes kept.
-- check region:942 18:65843454-65945591_:+_gene_943, 199556 candidate probes
--- 80100 probes passed check_dic selection.
finish in 7.414s, 1206 probes kept.
-- check region:943 18:68933328-69000228_:+_gene_944, 128380 candidate probes
--- 51010 probes passed check_dic selection.
finish in 4.817s, 782 probes kept.
-- check region:944 18:74461676-74530383_:+_gene_945, 136324 candidate probes
--- 69174 probes passed check_dic selection.
fini

--- 5815 probes passed check_dic selection.
finish in 2.202s, 127 probes kept.


### save probes

In [147]:
overwrite_savefile = True 
if not os.path.exists(pb_designer.save_file) or overwrite_savefile:
    #pb_designer.plots()
    #pb_designer.save_csv()
    pb_designer.save_to_file()
print(f"-- number of probes kept: {len(pb_designer.kept_probes)}")

- Save reports into file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr
-- number of probes kept: 821825


In [151]:
# after finish designing, remove all loaded OT maps to freeup memory usage
pb_designer.release_OTmaps()

Time to release OTmaps: 4.199s. 


In [152]:
pb_designer.cand_probes.items()

In [150]:
for _pb, _info in pb_designer.cand_probes.items():
    if _info['reg_name'][-1] == '_':
        old_name = str(_info['reg_name'])
        new_name = _info['reg_name'].replace(':+', 'strand_+') + str(_info['reg_index'])
        _info['name'] = _info['name'].replace(old_name, new_name)
        _info['reg_name'] = new_name

## 2.3 summarize into a pb_dict

start over from here is possible

In [6]:
# filename to save probe reports
save_file = os.path.join(report_folder, f'merged_probes.pbr')
print(save_file)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr


In [ ]:
_pbde_from_pickel = pickle.load(open(save_file,'rb'))

#_pbde_from_pickel = pickle.load(save_file)

In [ ]:
reload(library_tools)
reload(library_tools.design)

verbose=True
cand_prob_nums = []
pb_dict = {}
# Parse segemented list
#_pbde = library_tools.design.pb_reports_class()
#_pbde.load_from_file(save_file)



- Loading from savefile: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\mop_SE_1000\reports\merged_probes.pbr.


In [ ]:
_region_gid = list(_sel_pbs.values())[0]['reg_name'].split('_')[-1]
cand_prob_nums.append(len(_sel_pbs))
if verbose:
    print(f"-- appending gene:{_gene_name} with {len(_sel_pbs)} probes")
_splitted_pbde = ld.pb_reports_class()
_splitted_pbde.kept_probes = _sel_pbs
pb_dict[_gene_name] = _splitted_pbde